# Skyjo Player
A notebook which uses the SKyjo game file to execute a game of skyjo between two opponents. Able to play any two opponents against each other. Uses a custom game loop to allow player interaction if desired.

In [1]:
import pygame
from typing import List
from bots import Random
from game import Skyjo, Player, Card

pygame 2.1.0 (SDL 2.0.16, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Let's first run a test on the game with two Random bots to verify the game is operating correctly.

In [2]:
wins = {}
for _ in range(100):
    player_1 = Random("Random 1")
    player_2 = Random("Random 2")
    skyjo = Skyjo([player_1, player_2])
    winner = skyjo.play().name
    if winner not in wins.keys():
        wins[winner] = 0
    wins[winner] += 1

print(wins)

{'Random 1': 56, 'Random 2': 44}


### CardObject
A wrapper on the original Card class which adds logic for rendering to the screen.

In [3]:
class CardObject(Card):
    def __init__(self, card: Card, index: int, pos: List[int], size: List[int], identity=None):
        super().__init__(card.value, card.visible)
        self.card = card
        self.pos = pos
        self.size = size
        self.rect = pygame.Rect(pos, size)
        self.index = index

    def _hex_to_rgb(self, hex_code):
        hex_code = hex_code.lstrip('#')
        return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))

    def draw(self, screen):
        if self.card.visible:
            pygame.draw.rect(screen, self._hex_to_rgb(self.card.color), self.rect)
        else:
            pygame.draw.rect(screen, (195, 100, 230), self.rect)

        pygame.draw.rect(screen, (0, 0, 0), self.rect, 2)
        if str(self.card) != "None":
            font = pygame.font.Font(None, 72)
            text = font.render(str(self.card), True, (0, 0, 0))
            screen.blit(text, (self.pos[0] + 25 if len(str(self.card)) == 2 else self.pos[0] + 37.5 , self.pos[1] + 42.5))
        else:
            font = pygame.font.Font(None, 36)
            text = font.render("Skyjo", True, (0, 0, 0))
            screen.blit(text, (self.pos[0] + 17.5, self.pos[1] + 50))

### Initialization and Game Loop
Logic needed to set up and run the game. Set player1 and player2 for any type of player. Will assume that when playable=True, that the user wishes to play. All others will require automated code via the Player structure to execute.

In [4]:
# Players
player1 = Player("Player 1", True)
# player1 = Random("Random 1")
player2 = Random("Random 2")

# Pygame initialization
pygame.init()
screen = pygame.display.set_mode((960, 700))
clock = pygame.time.Clock()
running = True
deck = CardObject(Card(0), 0, [720, 575], [100, 140])
discard = CardObject(Card(0), 0, [830, 575], [100, 140])
selected = CardObject(Card(0), 0, [610, 575], [100, 140])

# Skyjo initialization
skyjo = Skyjo([player1, player2])
action = 'draw'
first_out = None

In [5]:
# Main game loop
while running:
    # Reset to false each time for automated players
    played = False

    # Get the latest cards in the player's hand
    player1_cards = [CardObject(card, i, [25 + (i % 4) * 110, 100 + (i // 4) * 160], [100, 140]) for i, card in enumerate(player1.hand.values()) if isinstance(card, Card)]
    player2_cards = [CardObject(card, i, [500 + (i % 4) * 110, 100 + (i // 4) * 160], [100, 140]) for i, card in enumerate(player2.hand.values()) if isinstance(card, Card)]

    # Check for player input (can interfer with automated players, so only interact with own cards if it's the player's turn)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # Check if the player clicked on the deck, discard (draw from), or selected card (discard the drawn card)
            if deck.rect.collidepoint(event.pos):
                if action == 'draw':
                    card = skyjo.deck.draw()
                    skyjo.turn.drawn_card = card
                    card.flip()
                    action = 'keep-discard'
            elif discard.rect.collidepoint(event.pos):
                if action == 'draw' and len(skyjo.discard) > 0:
                    card = skyjo.discard.draw()
                    skyjo.turn.drawn_card = card
                    action = 'keep-discard'
            elif selected.rect.collidepoint(event.pos):
                if action == 'keep-discard':
                    skyjo.discard.discard(skyjo.turn.drawn_card)
                    skyjo.turn.drawn_card = None
                    action = 'board'

            # Check which card of the player's own cards have been clicked (place card if one is drawn, flip is one isn't drawn. Perform turn cleanup at the end)
            # Player 1 logic
            if skyjo.turn == player1:
                for card in player1_cards:
                    if card.rect.collidepoint(event.pos):
                        if action == 'board' or action == 'keep-discard':
                            if skyjo.turn.drawn_card is not None:
                                old_card = skyjo.turn.replace_card(card.index, skyjo.turn.drawn_card)
                                skyjo.discard.discard(old_card)
                                skyjo.turn.drawn_card = None

                                # Turn cleanup
                                skyjo.check_for_column()
                                skyjo.turn.compute_score()
                                if first_out is not None:
                                    action = 'end'
                                else:
                                    action = 'draw'
                                skyjo.turn = player2
                                played = True

                            elif not card.card.visible:
                                card.card.flip()

                                # Turn cleanup
                                skyjo.check_for_column()
                                skyjo.turn.compute_score()
                                if first_out is not None:
                                    action = 'end'
                                else:
                                    action = 'draw'
                                skyjo.turn = player2
                                played = True

            # Player 2 logic
            if skyjo.turn == player2:
                for card in player2_cards:
                    if card.rect.collidepoint(event.pos):
                        if action == 'board' or action == 'keep-discard':
                            if skyjo.turn.drawn_card is not None:
                                old_card = skyjo.turn.replace_card(card.index, skyjo.turn.drawn_card)
                                skyjo.discard.discard(old_card)
                                skyjo.turn.drawn_card = None

                                # Turn cleanup
                                skyjo.check_for_column()
                                skyjo.turn.compute_score()
                                if first_out is not None:
                                    action = 'end'
                                else:
                                    action = 'draw'
                                skyjo.turn = player1
                                played = True

                            elif not card.card.visible:
                                card.card.flip()

                                # Turn cleanup
                                skyjo.check_for_column()
                                skyjo.turn.compute_score()
                                if first_out is not None:
                                    action = 'end'
                                else:
                                    action = 'draw'
                                skyjo.turn = player1
                                played = True

    # Automated player actions
    if skyjo.turn.playable == False:
        if action == 'draw' and not played:
            played = True
            result = skyjo.turn.draw_card(skyjo.players, skyjo.discard)
            # print("deck" if result == 0 else "discard")
            if result == 0:
                card = skyjo.deck.draw()
                skyjo.turn.drawn_card = card
                card.flip()
            else:
                card = skyjo.discard.draw()
                skyjo.turn.drawn_card = card
            action = 'keep-discard'
        elif action == 'keep-discard' and not played:
            played = True
            result = skyjo.turn.keep_discard_card(skyjo.players)
            # print(("discard" if result == 0 else "keep") + " card: " + str(skyjo.turn.drawn_card))
            if result == 0:
                skyjo.discard.discard(skyjo.turn.drawn_card)
                skyjo.turn.drawn_card = None
            action = 'board'
        elif action == 'board' and not played:
            played = True
            if skyjo.turn.drawn_card is not None:
                legal_actions = [i for i, card in skyjo.turn.hand.items() if card is not None]
            else:
                # Get the indexes of all cards in the player's hand which are not visible
                legal_actions = [i for i, card in skyjo.turn.hand.items() if card is not None and not card.visible]
            result = skyjo.turn.board_choice(skyjo.players, legal_actions)
            # print(result)
            if skyjo.turn.drawn_card == None:
                skyjo.turn.flip_card(result)
            else:
                old_card = skyjo.turn.replace_card(result, skyjo.turn.drawn_card)
                skyjo.discard.discard(old_card)
                skyjo.turn.drawn_card = None
            skyjo.check_for_column()
            skyjo.turn.compute_score()
            skyjo.turn = player1 if skyjo.turn == player2 else player2
            if first_out is not None:
                action = 'end'
            else:
                action = 'draw'

    # Draw the game state
    screen.fill("light blue")

    # Current player
    font = pygame.font.Font(None, 36)
    text = font.render(str(skyjo.turn) + "'s turn", True, (0, 0, 0))
    screen.blit(text, (400, 30))

    # Player 1's total score
    font = pygame.font.Font(None, 24)
    text = font.render("Total: " + str(player1.total_score), True, (0, 0, 0))
    screen.blit(text, (220, 35))

    # Player 1's round score
    font = pygame.font.Font(None, 36)
    text = font.render(str(player1) + ": " + str(player1.score), True, (0, 0, 0))
    screen.blit(text, (190, 60))

    # Player 2's total score
    font = pygame.font.Font(None, 24)
    text = font.render("Total: " + str(player2.total_score), True, (0, 0, 0))
    screen.blit(text, (680, 35))

    # Player 2's round score
    font = pygame.font.Font(None, 36)
    text = font.render(str(player2) + ": " + str(player2.score), True, (0, 0, 0))
    screen.blit(text, (650, 60))

    # Draw the cards
    for card in player1_cards + player2_cards:
        card.draw(screen)

    # Draw the deck if there are cards left
    if len(skyjo.deck) > 0:
        deck = CardObject(Card(0), 0, [720, 575], [100, 140])
        deck.draw(screen)

    # Draw the discard if there are cards left
    if len(skyjo.discard) > 0:
        discard = CardObject(skyjo.discard.cards[-1], 0, [830, 575], [100, 140])
        discard.draw(screen)

    # Draw the selected card if there is one
    if skyjo.turn.drawn_card is not None:
        selected = CardObject(skyjo.turn.drawn_card, 0, [610, 575], [100, 140])
        selected.draw(screen)

    # Check if either player has flipped over all their cards and set last_turn to True
    if all([str(card) != "None" for card in player1.hand.values()]) or all([str(card) != "None" for card in player2.hand.values()]):
        first_out = player1 if all([str(card) != "None" for card in player1.hand.values()]) else player2

    # Check if the round is over and reset the round
    if action == 'end':
        skyjo.last_winner = first_out
        skyjo.reset_round()
        first_out = None
        action = 'draw'

    # Check if the game is over and display the winner
    if player1.total_score >= 100 or player2.total_score >= 100:
        print(str(player1 if player1.total_score < player2.total_score else player2) + " wins!")
        break

    pygame.display.flip()
    clock.tick(60)

pygame.quit()